<a href="https://colab.research.google.com/github/veritaem/DS-Unit-2-Sprint-4-Model-Validation/blob/master/day_2_sprint_4_unit_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#imports and installs

In [1]:
!pip install category_encoders

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, classification_report, roc_auc_score, recall_score, precision_score
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from mlxtend.plotting import plot_decision_regions
from sklearn.pipeline import make_pipeline
import seaborn as sns


In [4]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip bank-additional.zip

--2019-01-30 02:46:11--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444572 (434K) [application/zip]
Saving to: ‘bank-additional.zip.1’

bank-additional.zip 100%[===================>] 434.15K   605KB/s    in 0.7s    

2019-01-30 02:46:12 (605 KB/s) - ‘bank-additional.zip.1’ saved [444572/444572]

Archive:  bank-additional.zip
replace bank-additional/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/bank-additional/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace bank-additional/.Rhistory? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace bank-additional/bank-additional-full.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace bank-additional/bank-additional-names.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [5]:
%cd bank-additional

/content/bank-additional


In [29]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df = pd.read_csv('bank-additional-full.csv', sep = ';')
print(df.shape)
df.head()

(41188, 21)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [38]:
df.groupby('euribor3m')['y'].nunique()

euribor3m
0.634    2
0.635    2
0.636    2
0.637    2
0.638    2
0.639    2
0.640    2
0.642    2
0.643    2
0.644    2
0.645    2
0.646    2
0.649    2
0.650    2
0.651    2
0.652    2
0.653    2
0.654    2
0.655    2
0.659    2
0.663    2
0.668    2
0.672    2
0.677    2
0.682    2
0.683    2
0.684    2
0.685    2
0.688    1
0.690    2
0.692    2
0.695    2
0.697    2
0.699    2
0.700    2
0.701    2
0.702    2
0.704    2
0.706    2
0.707    2
0.708    2
0.709    2
0.710    2
0.711    2
0.712    2
0.713    2
0.714    2
0.715    2
0.716    2
0.717    2
0.718    2
0.719    2
0.720    2
0.721    2
0.722    2
0.723    2
0.724    2
0.727    2
0.728    2
0.729    2
0.730    2
0.731    2
0.732    2
0.733    2
0.735    2
0.737    2
0.739    2
0.740    2
0.741    2
0.742    2
0.743    2
0.744    2
0.748    2
0.749    2
0.750    2
0.752    2
0.753    2
0.754    2
0.755    2
0.761    2
0.762    2
0.766    2
0.767    2
0.768    2
0.770    2
0.771    2
0.773    2
0.778    2
0.781    2
0.782    2


#pipeline

In [0]:
pipeline = make_pipeline(
                          ce.OneHotEncoder(use_cat_names = True),
                          StandardScaler(),
                          LogisticRegression())

# git a uh baseline

In [8]:
df.y.replace({'yes':1, 'no':0}, inplace = True)
df.y.mean()

0.11265417111780131

In [9]:
baseline = [df.y.mean()] * len(df)
mean_squared_error(df.y, baseline)

0.09996320884756243

In [10]:
df.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [11]:
X = df.drop(['y'], axis = 1)
y = df.y
X_train, X_test, y_train, y_test = train_test_split(X, y)
majority_class = y_train.mode()[0]
y_pred = np.full(fill_value =majority_class, shape = y_test.shape)
accuracy_score(y_test, y_pred)

0.8878314072059823

In [12]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.9124016703894338

Ok... 88.3% and 90.2% for non pipelined and pipelined, respectively... thats gonna be hard to beat!

#making the model

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle = True)
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

((23168, 20), (10297, 20), (7723, 20), (23168,), (10297,), (7723,))

In [14]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
accuracy_score(y_val, y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.9088437135828046

In [15]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [0]:
def three_way_split(
    X, y, train_size=0.8, val_size=0.1, test_size=0.1, 
    random_state=None, shuffle=False):
        
    assert train_size + val_size + test_size == 1
    
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=shuffle)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=val_size/(train_size+val_size), 
        random_state=random_state, shuffle=shuffle)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [0]:
X_train, X_val, X_test, y_train, y_val, y_test = three_way_split(
    X, y, shuffle=False)

In [22]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
print(accuracy_score(y_val, y_pred))

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(accuracy_score(y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.85360524399126


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5722262685117747


/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


ok no dice... time to reencode all this stuff i guess

#new variables

In [0]:
dur_bins = [0, 60, 120, 240, 480, 960, 1800, 3600, 100000]
dur_labels = ['<1min', '<2mins', '<4mins', '<8 mins', '<16mins', 'half hour', 'hour', 'hour+']
age_bins = [18, 25, 35, 45, 55, 65, 100]
age_labels = ['18-25', '25-35', '35-45', '45-55', '55-65', '65+']

df['age bins'] = pd.cut(df['age'], bins =age_bins, labels = age_labels)
df['dur_bins'] = pd.cut(df['duration'], bins = dur_bins, labels = dur_labels)

In [0]:
cpi_bins = [92, 92.5, 93, 93.5, 94, 94.5, 95]
cpi_labels = ['<92.5', '<93', '<93.5', '<94', '<94.5', '<95']
df['cpi'] = pd.cut(df['cons.price.idx'], bins = cpi_bins, labels = cpi_labels)

In [0]:
cci_bins = [26, 30, 35, 40, 45, 50 , 55]
cci_labels = ['25-30', '30-35', '35-40', '40-45', '45-50', '50+']
df['cci'] = pd.cut(df['cons.conf.idx'], bins = cci_bins, labels = cci_labels)

In [42]:
df.groupby('cons.conf.idx')['y'].nunique()

cons.conf.idx
-50.8    2
-50.0    2
-49.5    2
-47.1    2
-46.2    2
-45.9    2
-42.7    2
-42.0    2
-41.8    2
-40.8    2
-40.4    2
-40.3    2
-40.0    2
-39.8    2
-38.3    2
-37.5    2
-36.4    2
-36.1    2
-34.8    2
-34.6    2
-33.6    2
-33.0    2
-31.4    2
-30.1    2
-29.8    2
-26.9    2
Name: y, dtype: int64

In [0]:
X = df.drop(['duration', 'contact', 'day', 'age', 'duration', 'y'], axis = 1)

In [0]:
X_train, X_val, X_test, y_train, y_val, y_test = three_way_split(
    X, y, shuffle=False)

In [0]:
X_train

In [0]:
y_train

In [0]:

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
print(accuracy_score(y_val, y_pred))
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [0]:
y.rolling(5000).mean().plot();